In [1]:
import csv
import numpy as np
import sys
import pandas as pd
import itertools
import math
import time

from sklearn import svm, linear_model, neighbors
from sklearn import tree, ensemble
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB

import networkx as nx
import random
import numbers

from sklearn.model_selection import StratifiedKFold

from src import ml

In [2]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext

In [3]:
if False: 
    sc.stop()

config = SparkConf()
config.setMaster("local")
config.set("spark.executor.memory", "15g")
config.set('spark.driver.memory', '20g')
config.set("spark.memory.offHeap.enabled",True)
config.set("spark.memory.offHeap.size","15g") 
sc = SparkContext(conf=config)
print (sc)

your 131072x1 screen size is bogus. expect trouble
23/09/05 11:12:11 WARN Utils: Your hostname, Rz resolves to a loopback address: 127.0.1.1; using 172.17.110.41 instead (on interface eth0)
23/09/05 11:12:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/05 11:12:13 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local appName=pyspark-shell>


# DRUGBANK 4.0

In [4]:
ddi_df = pd.read_csv("data/input/ddi_v4.txt", sep='\t')
ddi_df.head()

,Drug1,Drug2
0,DB00005,DB00026
1,DB00026,DB00005
2,DB00005,DB00065
3,DB00065,DB00005
4,DB00005,DB00072


In [5]:
featureFilename = "vectors/DB/RDF2Vec_sg_200_5_5_15_2_500_d5_uniform.txt"
embedding_df = pd.read_csv(featureFilename, delimiter='\t') 
embedding_df.Entity =embedding_df.Entity.str[-8:-1]
embedding_df.rename(columns={'Entity':'Drug'}, inplace=True)

In [6]:
pairs, classes = ml.generatePairs(ddi_df, embedding_df)

In [7]:
nb_model = GaussianNB()
lr_model = linear_model.LogisticRegression()
rf_model = ensemble.RandomForestClassifier(n_estimators=200, n_jobs=-1)

#clfs = [('Naive Bayes',nb_model),('Logistic Regression',lr_model),('Random Forest',rf_model)]
clfs = [('Random Forest',rf_model)]

In [8]:
n_seed =100
n_fold = 10
n_run = 5
n_proportion = 1
all_scores_df = ml.kfoldCV(sc, pairs, classes, embedding_df, clfs, n_run, n_fold, n_proportion, n_seed)


+/-: 47077 47077 1015076
run 0


23/09/05 11:12:39 WARN TaskSetManager: Stage 0 contains a task of very large size (7363 KiB). The maximum recommended task size is 1000 KiB.
84738 9416                                                          (0 + 1) / 1]
84738 9416
84738 9416
84738 9416                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
23/09/05 11:17:14 WARN TaskSetManager: Stage 1 contains a task of very large size (7363 KiB). The maximum recommended task size is 1000 KiB.
84738 9416


+/-: 47077 47077 1015076
run 1


84738 9416                                                          (0 + 1) / 1]
84738 9416
84738 9416                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
23/09/05 11:21:55 WARN TaskSetManager: Stage 2 contains a task of very large size (7363 KiB). The maximum recommended task size is 1000 KiB.


+/-: 47077 47077 1015076
run 2


84738 9416
84738 9416                                                          (0 + 1) / 1]
84738 9416
84738 9416                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
23/09/05 11:26:24 WARN TaskSetManager: Stage 3 contains a task of very large size (7363 KiB). The maximum recommended task size is 1000 KiB.
84738 9416


+/-: 47077 47077 1015076
run 3


84738 9416                                                          (0 + 1) / 1]
84738 9416
84738 9416                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
23/09/05 11:30:53 WARN TaskSetManager: Stage 4 contains a task of very large size (7363 KiB). The maximum recommended task size is 1000 KiB.
84738 9416


+/-: 47077 47077 1015076
run 4


84738 9416                                                          (0 + 1) / 1]
84738 9416
84738 9416                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]
84739 9415
84739 9415                                                          (0 + 1) / 1]


In [ ]:
agg_df = all_scores_df.groupby(['method','run']).mean().groupby('method').mean()



In [ ]:
agg_df.to_csv('results/traditional/DB4_TCV_run_sg.csv',sep=',', index=False)

# TWOSIDES 

In [ ]:
ddi_df = pd.read_csv("data/input/ddi_twosides.txt", sep='\t')
ddi_df.head()

In [ ]:
len(ddi_df)

In [ ]:
len( set(ddi_df.Drug1.unique()).union(ddi_df.Drug2.unique()) )

In [ ]:
featureFilename = "vectors/DB/RDF2Vec_sg_200_5_5_15_2_500_d5_uniform.txt"
embedding_df = pd.read_csv(featureFilename, delimiter='\t') 
embedding_df.Entity =embedding_df.Entity.str[-8:-1]
embedding_df.rename(columns={'Entity':'Drug'}, inplace=True)

In [ ]:
pairs, classes = ml.generatePairs(ddi_df, embedding_df)

In [ ]:
n_seed =100
n_fold =10 
n_run =10 
n_proportion = 1,
all_scores_df = ml.kfoldCV(sc, pairs, classes, embedding_df, clfs, n_run, n_fold, n_proportion, n_seed)

In [ ]:
all_scores_df.groupby(['method','run']).mean().groupby('method').mean()

In [ ]:
all_scores_df.to_csv('results/traditional/TWOSIDES_TCV_run_sg.csv',sep=',', index=False)

# KEGG

In [ ]:
ddi_df = pd.read_csv("data/input/ddi_kegg.txt", sep='\t')
ddi_df.head()

In [ ]:
len(ddi_df)

In [ ]:
len( set(ddi_df.Drug1.unique()).union(ddi_df.Drug2.unique()) )

In [ ]:
featureFilename = "vectors/DB/RDF2Vec_sg_200_5_5_15_2_500_d5_uniform.txt"
embedding_df = pd.read_csv(featureFilename, delimiter='\t') 
embedding_df.Entity =embedding_df.Entity.str[-8:-1]
embedding_df.rename(columns={'Entity':'Drug'}, inplace=True)

In [ ]:
pairs, classes = ml.generatePairs(ddi_df, embedding_df)

In [ ]:
n_seed =100
n_fold =10 
n_run =10 
n_proportion = 1,
all_scores_df = ml.kfoldCV(sc, pairs, classes, embedding_df, clfs, n_run, n_fold, n_proportion, n_seed)

In [ ]:
all_scores_df.groupby(['method','run']).mean().groupby('method').mean()

In [ ]:
all_scores_df.to_csv('results/traditional/KEGG_TCV_run_sg.csv',sep=',', index=False)

# Drugbank 5

In [ ]:
ddi_df = pd.read_csv("data/input/ddi_v5.txt", sep='\t')
ddi_df.head()

In [ ]:
featureFilename = "vectors/DB/RDF2Vec_sg_200_5_5_15_2_500_d5_uniform.txt"
embedding_df = pd.read_csv(featureFilename, delimiter='\t') 
embedding_df.Entity =embedding_df.Entity.str[-8:-1]
embedding_df.rename(columns={'Entity':'Drug'}, inplace=True)

In [ ]:
len(ddi_df)

In [ ]:
len( set(ddi_df.Drug1.unique()).union(ddi_df.Drug2.unique()) )

In [ ]:
pairs, classes = ml.generatePairs(ddi_df, embedding_df)

In [ ]:
nb_model = GaussianNB()
logistic_model = linear_model.LogisticRegression(C=0.01)
rf_model = ensemble.RandomForestClassifier(n_estimators=200, n_jobs=-1)
clfs = [('Naive Bayes',nb_model),('Logistic Regression',lr_model),('Random Forest',rf_model)]

In [ ]:
n_seed =100
n_fold =10 
n_run =10 
n_proportion = 1,
all_scores_df = ml.kfoldCV(sc, pairs, classes, embedding_df, clfs, n_run, n_fold, n_proportion, n_seed)

In [ ]:
all_scores_df.groupby(['method','run']).mean().groupby('method').mean()

In [ ]:
all_scores_df.to_csv('results/traditional/DBv5_TCV_run_sg.csv',sep=',', index=False)

# Drugbankv5 - RDF2Vec CBOW 

In [ ]:
featureFilename = "vectors/DB/RDF2Vec_cbow_200_5_5_2_500_d5_uniform.txt"
embedding_df = pd.read_csv(featureFilename, delimiter='\t') 
embedding_df.Entity =embedding_df.Entity.str[-8:-1]
embedding_df.rename(columns={'Entity':'Drug'}, inplace=True)

In [ ]:
pairs, classes = ml.generatePairs(ddi_df, embedding_df)

In [ ]:
nb_model = GaussianNB()
logistic_model = linear_model.LogisticRegression(C=0.01)
rf_model = ensemble.RandomForestClassifier(n_estimators=200, n_jobs=-1)
clfs = [('Naive Bayes',nb_model),('Logistic Regression',lr_model),('Random Forest',rf_model)]

In [ ]:
n_seed =100
n_fold =10 
n_run =10 
n_proportion = 1,
all_scores_df = ml.kfoldCV(sc, pairs, classes, embedding_df, clfs, n_run, n_fold, n_proportion, n_seed)

In [ ]:
all_scores_df.groupby(['method','run']).mean().groupby('method').mean()

In [ ]:
all_scores_df.to_csv('results/traditional/DBv5_TCV_run_cbow.csv',sep=',', index=False)

# Drugbankv5 - TransE

In [ ]:
featureFilename = "vectors/DB/TransE_DRUGBANK_d_100.txt"
embedding_df = pd.read_csv(featureFilename, delimiter='\t') 

In [ ]:
pairs, classes = ml.generatePairs(ddi_df, embedding_df)

In [ ]:
n_seed =100
n_fold =10 
n_run =10 
n_proportion = 1,
all_scores_df = ml.kfoldCV(sc, pairs, classes, embedding_df, clfs, n_run, n_fold, n_proportion, n_seed)

In [ ]:
all_scores_df.groupby(['method','run']).mean().groupby('method').mean()

In [ ]:
all_scores_df.to_csv('results/traditional/DBv5_TCV_run_transe.csv',sep=',', index=False)

# Drugbankv5 - TransD

In [ ]:
featureFilename = "vectors/DB/TransD_DRUGBANK_d_100.txt"
embedding_df = pd.read_csv(featureFilename, delimiter='\t') 

In [ ]:
pairs, classes = ml.generatePairs(ddi_df, embedding_df)

In [ ]:
n_seed =100
n_fold =10 
n_run =10 
n_proportion = 1,
all_scores_df = ml.kfoldCV(sc, pairs, classes, embedding_df, clfs, n_run, n_fold, n_proportion, n_seed)

In [ ]:
all_scores_df.groupby(['method','run']).mean().groupby('method').mean()

In [ ]:
all_scores_df.to_csv('results/traditional/DBv5_TCV_run_transd.csv',sep=',', index=False)